In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os, re

# url = 'https://www.kaigokensaku.mhlw.go.jp/23/index.php?action_kouhyou_pref_search_search=true&method=search&p_count=50&p_offset=0'
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'}

# 従業員テーブルをリサーチする
def find_jyugyouin_df(df_list):
    for df in df_list:
        if '総従業者数' in df.to_string():
            return df

# 抽出データ格納用フォルダーを作成する
if not os.path.exists('data'):
    os.mkdir('data')

# 使用時に指定が必要なパラメータ
KEN = '35' # この数字を変更すれば、県違いでリサーチ可能
PAGES = 3 # ページ数を指定ください。0の場合、デフォルトは全ページとなる、各ページに50項目有り

# 総ページ数を取得する
url_page = 'https://www.kaigokensaku.mhlw.go.jp/{}/index.php?action_kouhyou_pref_search_search=true&method=search&p_count=50&p_offset=0'.format(KEN)
url_page_info = requests.get(url_page, headers=header)
total = json.loads(url_page_info.text)['pager']['total']
pages_ = int(total / 50) + 1
if PAGES == 0:
    pages = pages_
else:
    pages = PAGES

# 全体URLのリストを生成する
urls = ['https://www.kaigokensaku.mhlw.go.jp/{}/index.php?action_kouhyou_pref_search_search=true&method=search&p_count=50&p_offset={}'.format(KEN,i) for i in range(0,pages*50,50)] # pages*50
for i,url in enumerate(urls):
    infos = []
    print('スクレイピング中:{}/{}'.format(i+1,pages))
    webInfo = requests.get(url, headers=header)
    jsonInfo = json.loads(webInfo.text)
    data = jsonInfo['data']
    for item in data:
        print('   {}...'.format(item.get('JigyosyoName')))
        tmp = {
            'JigyosyoName': item.get('JigyosyoName'),
            'JigyosyoJyusho':item.get('JigyosyoJyusho'),
            'JigyosyoTel':item.get('JigyosyoTel'),
            'JigyosyoCd':item.get('JigyosyoCd'),
            'JigyosyoSubCd':item.get('JigyosyoSubCd'),
            'ServiceCd':item.get('ServiceCd'),
            'VersionCd':item.get('VersionCd'),
            'JigyosyoYubinbangou':item.get('JigyosyoYubinbangou').replace(',','-')
        }

        # 法人名称を取得する
        houjin_url = 'https://www.kaigokensaku.mhlw.go.jp/{}/index.php?action_kouhyou_detail_004_kihon=true&JigyosyoCd={}-{}&ServiceCd={}'.format(
            KEN, tmp['JigyosyoCd'], tmp['JigyosyoSubCd'], tmp['ServiceCd'])
        houjinInfo = requests.get(houjin_url, headers=header)
        houjinbs = BeautifulSoup(houjinInfo.text, 'lxml')
        n = houjinbs.select('.houjin_number_check')
        tmp['houjin'] = n[1].text


        # 従業者情報を取得する
        jyugyouin_url = 'https://www.kaigokensaku.mhlw.go.jp/{}/index.php?action_kouhyou_detail_004_kani=true&JigyosyoCd={}-{}&ServiceCd={}'.format(
            KEN, tmp['JigyosyoCd'], tmp['JigyosyoSubCd'], tmp['ServiceCd'])
        jyugyouinInfo = requests.get(jyugyouin_url, headers =header)
        if not os.path.exists('data/{}'.format(tmp['JigyosyoName'])):
            os.mkdir(f"data/{tmp['JigyosyoName']}")
        df_table_jyugyouin = pd.read_html(jyugyouinInfo.text)
        df_jyugyouin = find_jyugyouin_df(df_table_jyugyouin)
        df_jyugyouin.to_csv('data/{}/従業者.csv'.format(tmp['JigyosyoName']), encoding='cp932')

        # 運営タブにあるすべてのテーブルを取得する
        unei_url = 'https://www.kaigokensaku.mhlw.go.jp/{}/index.php?action_kouhyou_detail_004_unei=true&JigyosyoCd={}-{}&ServiceCd={}'.format(
            KEN, tmp['JigyosyoCd'], tmp['JigyosyoSubCd'], tmp['ServiceCd'])
        uneiInfo = requests.get(unei_url, headers=header)
        temp_unei = uneiInfo.text
        pattern = re.compile(r"<tr class='isEtcRow'>.*?</tr>",re.S)
        temp_unei = re.sub(pattern,'',temp_unei)
        temp_unei = re.sub(r'<img width="32" height="32" alt="-" .*?>', '-', temp_unei)
        temp_unei = re.sub(r'<img width="32" height="32" alt="○" .*?>', '○', temp_unei)
        df_list = pd.read_html(temp_unei)
        labels = [
            '1．利用者の権利擁護のための取組',
            '2．利用者本位の介護サービスの提供',
            '3．相談、苦情等の対応のために講じている措置',
            '4．サービスの内容の評価や改善等',
            '5．サービスの質の確保、透明性の確保等のための外部機関等との連携',
            '6．適切な事業運営の確保',
            '7．事業所の運営管理、業務分担、情報の共有等',
            '8．安全管理及び衛生管理',
            '9．情報の管理、個人情報保護等',
            '10．その他、介護サービスの質の確保のために行っていること'
        ]
        for i,item in enumerate(df_list):
            if i==0:
                continue
            item.drop(item.columns[1:3], axis=1, inplace=True)
            item.to_csv('data/{}/{}.csv'.format(tmp['JigyosyoName'],labels[i-1]), encoding='cp932')

        infos.append(tmp)
    df = pd.DataFrame(infos)
    df.to_csv('info.csv',mode='a',header=None, encoding='cp932')
print('スクレイピングは完了しました！')


/Users/watanabewataru1/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


スクレイピング中:1/3
   (有)ホワイトクリーニング萩　ケアホワイト...
   (株)ナカニシ...
   24時間ケアサポートよしみ・安岡...
   24時間宅老所　楽さん家...
   24時間訪問サービスしものせき...
   Hot Spring...
   JAケアビレッジ・デイ新南陽...
   JAデイサービス久保...
   JA小規模多機能おひさま下松...
   JA山口県デイサービス下関...
   JA山口県デイサービス美祢...
   JA山口県介護支援センター美祢...
   JA訪問介護ふれあい徳山...
   JCHO徳山中央病院附属訪問看護ステーション...
   JＡ訪問介護ふれあい新南陽...
   TryQ（トライク）デイフィットネス...
   acotデイサービスセンター...
   あいあい山口...
   あいあい山口デイケア...
   あいあい山口短期入所生活介護...
   あいおい苑居宅介護支援事業所...
   あいらの杜・新岩国駅前...
   あい・周南ケアセンター...
   あい・周南ケアセンター...
   あおぞらの里　下関ケアプランセンター...
   あおぞらの里　下関デイサービスセンター...
   あおぞらの里　下関幡生デイサービスセンター...
   あおぞらの里　下関訪問看護ステーション...
   あおぞらの里　西の端ケアプランセンター...
   あおぞら居宅介護支援事業所...
   あおぞら居宅介護支援事業所...
   あさひデイサービスセンター則貞...
   あじさい居宅介護支援事業所...
   あすとぴあ光栄在宅介護支援センター...
   あそか苑デイサービスセンター...
   あまの訪問看護ステーション...
   あらなほーむ...
   ありがとうやない居宅介護支援事業所...
   あんの循環器内科　通所リハビリテーション...
   いこいの郷　白鳩...
   いしいケア・クリニック...
   いしいケア・クリニック...
   いしいケア・クリニック介護保険相談室...
   いろは詩デイサービスセンター...
   いろは詩居宅介護支援事業所...
   えがおデイサービスセンター...
   おうかデイサービスセンター...
   お